<a href="https://colab.research.google.com/github/birulboy/estructura_de_datos/blob/master/EstructuraDeDatosKmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📍 K-Means

K-Means es un algoritmo de *clustering* no supervisado que agrupa datos en **k** clústeres.

1. Se eligen k puntos aleatorios como centroides.
2. Cada dato se asigna al centroide más cercano (formando clústeres).
3. Se recalculan los centroides como el promedio de los puntos en cada clúster.
4. Se repite el proceso hasta que los centroides no cambian o se alcanza un límite de iteraciones.

El objetivo es minimizar la distancia total entre los puntos y su centroide correspondiente.

Usos comunes: segmentación de clientes, compresión de imágenes, detección de patrones.


In [5]:
%%writefile kmeans.cpp
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <cmath>
#include <cstdlib>
#include <ctime>
#include <limits>

using namespace std;

class Point
{
public:
    vector<double> features;
    int cluster_id;

    Point(const vector<double> &f) : features(f), cluster_id(-1) {}
};

class KMeans
{
private:
    int k;
    int max_iters;
    vector<vector<double>> centroids;

    double euclideanDistance(const vector<double> &a, const vector<double> &b)
    {
        double sum = 0.0;
        for (size_t i = 0; i < a.size(); ++i)
            sum += pow(a[i] - b[i], 2);
        return sqrt(sum);
    }

    void assignClusters(vector<Point> &points)
    {
        for (auto &point : points)
        {
            double min_dist = numeric_limits<double>::max();
            int best_cluster = -1;
            for (int i = 0; i < k; ++i)
            {
                double dist = euclideanDistance(point.features, centroids[i]);
                if (dist < min_dist)
                {
                    min_dist = dist;
                    best_cluster = i;
                }
            }
            point.cluster_id = best_cluster;
        }
    }

    void updateCentroids(vector<Point> &points)
    {
        vector<vector<double>> new_centroids(k, vector<double>(points[0].features.size(), 0.0));
        vector<int> counts(k, 0);

        for (const auto &point : points)
        {
            for (size_t j = 0; j < point.features.size(); ++j)
                new_centroids[point.cluster_id][j] += point.features[j];
            counts[point.cluster_id]++;
        }

        for (int i = 0; i < k; ++i)
        {
            if (counts[i] == 0)
                continue;
            for (size_t j = 0; j < new_centroids[i].size(); ++j)
                new_centroids[i][j] /= counts[i];
        }

        centroids = new_centroids;
    }

public:
    KMeans(int k, int max_iters) : k(k), max_iters(max_iters) {}

    void fit(vector<Point> &points)
    {
        centroids.clear();
        srand(time(0));
        for (int i = 0; i < k; ++i)
        {
            int index = rand() % points.size();
            centroids.push_back(points[index].features);
        }

        for (int iter = 0; iter < max_iters; ++iter)
        {
            assignClusters(points);
            updateCentroids(points);
        }
    }

    void printResults(const vector<Point> &points)
    {
        vector<int> cluster_counts(k, 0);

        for (const auto &point : points)
        {
            cluster_counts[point.cluster_id]++;
            cout << "Cluster " << point.cluster_id << ": ";
            for (double val : point.features)
                cout << val << " ";
            cout << endl;
        }

        cout << "\n--- Resumen por Clúster ---\n";
        for (int i = 0; i < k; ++i)
        {
            cout << "Clúster " << i << ": " << cluster_counts[i] << " elementos\n";
        }

        cout << "\n--- Centroides Finales ---\n";
        for (int i = 0; i < k; ++i)
        {
            cout << "Clúster " << i << " centroide: ";
            for (double val : centroids[i])
                cout << val << " ";
            cout << endl;
        }
    }
};

vector<Point> loadData(const string &filename)
{
    ifstream file(filename);
    string line;
    vector<Point> data;

    bool header_skipped = false;

    while (getline(file, line))
    {
        if (!header_skipped)
        {
            header_skipped = true;
            continue;
        }

        stringstream ss(line);
        string cell;
        vector<double> features;
        int col = 0;

        while (getline(ss, cell, '\t'))
        {
            // Column indices: Income = 4, MntWines = 9 to MntGoldProds = 14
            if (col == 4 || (col >= 9 && col <= 14))
            {
                try
                {
                    double val = cell.empty() ? 0.0 : stod(cell);
                    features.push_back(val);
                }
                catch (...)
                {
                    features.push_back(0.0); // fallback for non-numeric or missing values
                }
            }
            col++;
        }

        if (!features.empty())
        {
            data.emplace_back(features);
        }
    }

    return data;
}

int main()
{
    vector<Point> data = loadData("kmeans_cleaned.txt");
    if (data.empty())
    {
        cerr << "No data loaded. Verifica el archivo.\n";
        return 1;
    }

    KMeans kmeans(3, 100);
    kmeans.fit(data);
    kmeans.printResults(data);

    return 0;
}

Overwriting kmeans.cpp


In [6]:

!wget -O kmeans_cleaned.txt https://raw.githubusercontent.com/birulboy/estructura_de_datos/215684e9ec0e938ac08f55f05878801bdd728be4/Graph/kmeans_cleaned.txt


!g++ kmeans.cpp -o kmeans_exec


!./kmeans_exec


--2025-05-21 15:00:46--  https://raw.githubusercontent.com/birulboy/estructura_de_datos/215684e9ec0e938ac08f55f05878801bdd728be4/Graph/kmeans_cleaned.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58643 (57K) [text/plain]
Saving to: ‘kmeans_cleaned.txt’

kmeans_cleaned.txt  100%[===================>]  57.27K  --.-KB/s    in 0.003s  

2025-05-21 15:00:46 (19.3 MB/s) - ‘kmeans_cleaned.txt’ saved [58643/58643]

Cluster 2: 172 
Cluster 1: 2 
Cluster 0: 111 
Cluster 1: 10 
Cluster 0: 46 
Cluster 1: 0 
Cluster 0: 50 
Cluster 1: 3 
Cluster 1: 3 
Cluster 1: 1 
Cluster 1: 11 
Cluster 2: 225 
Cluster 1: 3 
Cluster 1: 6 
Cluster 0: 59 
Cluster 1: 2 
Cluster 2: 150 
Cluster 1: 0 
Cluster 1: 30 
Cluster 0: 69 
Cluster 1: 1 
Cluster 1: 0 
Cluster 1: 21 
Cluster 1: 39

# 🌐 Algoritmo de Prims

Prim encuentra el **árbol de expansión mínima** de un grafo conexo y no dirigido.

1. Se empieza desde un nodo arbitrario.
2. En cada paso, se elige la arista de menor peso que conecta un nodo del árbol con uno que aún no está en él.
3. Se repite hasta incluir todos los nodos.

Siempre elige el camino más barato posible sin formar ciclos.  
Ideal para diseñar redes eficientes (electricidad, internet, caminos).


In [7]:
%%writefile prims.cpp
#include <iostream>
#include <vector>
#include <limits>
#include <random>
using namespace std;

class Graph
{
private:
    vector<vector<int>> adj_matrix;

public:
    Graph(int n)
    {
        adj_matrix = vector<vector<int>>(n, vector<int>(n, -1));
    }

    void add_edge(int u, int v, int weight)
    {
        if (u == v)
            return; // evitar bucles
        adj_matrix[u][v] = weight;
        adj_matrix[v][u] = weight;
    }

    bool has_edge(int u, int v)
    {
        return adj_matrix[u][v] != -1;
    }

    void print()
    {
        cout << "Matriz de adyacencia ponderada:\n";
        int n = adj_matrix.size();
        for (int i = 0; i < n; i++)
        {
            for (int j = 0; j < n; j++)
            {
                if (adj_matrix[i][j] == -1)
                    cout << ". ";
                else
                    cout << adj_matrix[i][j] << " ";
            }
            cout << endl;
        }
    }

    void prim()
    {
        int n = adj_matrix.size();
        vector<int> parent(n, -1);                      // Almacena el MST
        vector<int> key(n, numeric_limits<int>::max()); // Coste mínimo para conectar
        vector<bool> inMST(n, false);                   // Nodo incluido en el MST

        key[0] = 0; // Empezamos desde el nodo 0

        for (int count = 0; count < n - 1; ++count)
        {
            // Encontrar el nodo u con menor key[] no incluido aún
            int u = -1;
            int min_key = numeric_limits<int>::max();
            for (int v = 0; v < n; ++v)
            {
                if (!inMST[v] && key[v] < min_key)
                {
                    min_key = key[v];
                    u = v;
                }
            }

            if (u == -1)
                break; // grafo no conexo

            inMST[u] = true;

            // Actualizar claves y padres de los nodos adyacentes
            for (int v = 0; v < n; ++v)
            {
                if (adj_matrix[u][v] != -1 && !inMST[v] && adj_matrix[u][v] < key[v])
                {
                    key[v] = adj_matrix[u][v];
                    parent[v] = u;
                }
            }
        }

        cout << "\nAristas del Árbol de Expansión Mínima (Prim):\n";
        int total_weight = 0;
        for (int i = 1; i < n; ++i)
        {
            if (parent[i] != -1)
            {
                cout << parent[i] << " - " << i << " (peso " << adj_matrix[i][parent[i]] << ")\n";
                total_weight += adj_matrix[i][parent[i]];
            }
        }
        cout << "Peso total del MST: " << total_weight << endl;
    }
};

int main()
{
    int n = 11;
    Graph g(n);

    // Generador de aristas aleatorias (con peso entre 1 y 20)
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> node_dist(0, n - 1);
    uniform_int_distribution<> weight_dist(1, 20);

    int edges_added = 0;
    while (edges_added < 20)
    {
        int u = node_dist(gen);
        int v = node_dist(gen);
        int w = weight_dist(gen);
        if (u != v && !g.has_edge(u, v))
        {
            g.add_edge(u, v, w);
            edges_added++;
        }
    }

    g.print();
    g.prim();

    return 0;
}


Writing prims.cpp


In [8]:
!g++ prims.cpp -o prims_exec


!./prims_exec

Matriz de adyacencia ponderada:
. . . . 15 . 9 5 7 . . 
. . . . . . 10 . 16 19 . 
. . . 12 . 20 . . 18 . . 
. . 12 . 3 7 . . . . 2 
15 . . 3 . . 5 17 . . . 
. . 20 7 . . . 9 . . . 
9 10 . . 5 . . . 8 16 . 
5 . . . 17 9 . . . 2 . 
7 16 18 . . . 8 . . . 12 
. 19 . . . . 16 2 . . . 
. . . 2 . . . . 12 . . 

Aristas del Árbol de Expansión Mínima (Prim):
6 - 1 (peso 10)
3 - 2 (peso 12)
4 - 3 (peso 3)
6 - 4 (peso 5)
3 - 5 (peso 7)
8 - 6 (peso 8)
0 - 7 (peso 5)
0 - 8 (peso 7)
7 - 9 (peso 2)
3 - 10 (peso 2)
Peso total del MST: 61


# 🕸️ Algoritmo de Kruskall

Kruskal también encuentra el **árbol de expansión mínima** de un grafo no dirigido.

1. Se ordenan todas las aristas de menor a mayor peso.
2. Se agregan al árbol aquellas que conectan nodos distintos y **no forman ciclos**.
3. Se detiene cuando todos los nodos están conectados.

Utiliza estructuras como conjuntos disjuntos (union-find) para evitar ciclos.  
Eficiente cuando el grafo tiene muchas aristas dispersas.

In [9]:
%%writefile kruskall.cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <random>
using namespace std;

class UnionFind {
private:
    vector<int> parent, rank;

public:
    UnionFind(int n) {
        parent.resize(n);
        rank.resize(n, 0);
        for (int i = 0; i < n; ++i)
            parent[i] = i;
    }

    int find(int x) {
        if (parent[x] != x)
            parent[x] = find(parent[x]);
        return parent[x];
    }

    bool union_sets(int x, int y) {
        int xr = find(x);
        int yr = find(y);
        if (xr == yr)
            return false;
        if (rank[xr] < rank[yr])
            parent[xr] = yr;
        else if (rank[xr] > rank[yr])
            parent[yr] = xr;
        else {
            parent[yr] = xr;
            rank[xr]++;
        }
        return true;
    }
};

class Graph {
private:
    vector<vector<int>> adj_matrix;
    vector<int> edges_u;
    vector<int> edges_v;
    vector<int> edges_weight;

public:
    Graph(int n) {
        adj_matrix = vector<vector<int>>(n, vector<int>(n, -1));
    }

    void add_edge(int u, int v, int weight) {
        adj_matrix[u][v] = weight;
        adj_matrix[v][u] = weight;
    }

    void print() {
        cout << "Matriz de adyacencia ponderada:\n";
        for (const auto& row : adj_matrix) {
            for (int val : row) {
                if (val == -1) cout << ". ";
                else cout << val << " ";
            }
            cout << endl;
        }
    }

    void extract_edges() {
        int n = adj_matrix.size();
        edges_u.clear();
        edges_v.clear();
        edges_weight.clear();

        for (int i = 0; i < n; ++i) {
            for (int j = i + 1; j < n; ++j) {
                if (adj_matrix[i][j] != -1) {
                    edges_u.push_back(i);
                    edges_v.push_back(j);
                    edges_weight.push_back(adj_matrix[i][j]);
                }
            }
        }
    }

    void kruskal() {
        extract_edges();
        int n = adj_matrix.size();
        int m = edges_u.size();


        vector<int> indices(m);
        for (int i = 0; i < m; ++i) indices[i] = i;

        sort(indices.begin(), indices.end(), [&](int a, int b) {
            return edges_weight[a] < edges_weight[b];
        });

        UnionFind uf(n);
        int mst_weight = 0;

        cout << "\nAristas del Árbol de Expansión Mínima (Kruskal):\n";
        for (int i : indices) {
            int u = edges_u[i];
            int v = edges_v[i];
            int w = edges_weight[i];

            if (uf.union_sets(u, v)) {
                cout << u << " - " << v << " (peso " << w << ")\n";
                mst_weight += w;
            }
        }
        cout << "Peso total del MST: " << mst_weight << endl;
    }
};

int main() {
    int n = 11;
    Graph g(n);

    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> distrib(0, n - 1);
    uniform_int_distribution<> weight_dist(1, 100);

    for (int i = 0; i < 10; i++) {
        int u = distrib(gen);
        int v = distrib(gen);
        while (v == u) v = distrib(gen);
        g.add_edge(u, v, weight_dist(gen));
    }

    g.print();
    g.kruskal();

    return 0;
}



Writing kruskall.cpp


In [10]:
!g++ kruskall.cpp -o kruskall_exec


!./kruskall_exec

Matriz de adyacencia ponderada:
. . . . . . . . . . . 
. . 8 53 . . . . 33 . . 
. 8 . . . . 13 . . . . 
. 53 . . . . . 5 . . . 
. . . . . . 19 . 71 . . 
. . . . . . . . . . . 
. . 13 . 19 . . . . 96 43 
. . . 5 . . . . . . . 
. 33 . . 71 . . . . . . 
. . . . . . 96 . . . . 
. . . . . . 43 . . . . 

Aristas del Árbol de Expansión Mínima (Kruskal):
3 - 7 (peso 5)
1 - 2 (peso 8)
2 - 6 (peso 13)
4 - 6 (peso 19)
1 - 8 (peso 33)
6 - 10 (peso 43)
1 - 3 (peso 53)
6 - 9 (peso 96)
Peso total del MST: 270
